# Pallier Encryption Demo
https://en.wikipedia.org/wiki/Paillier_cryptosystem

## Key Generation

In [2]:
import math

# choose p and q prime numbers
p = 7759
q = 6983

# calculate n and lamb
n = p * q
lamb = (p - 1) * (q - 1)

# simpler variant
g = n + 1

def extended_gcd(a, b):
    """Extended Euclidean Algorithm to find x such that ax ≡ 1 (mod b)."""
    if a == 0:
        return b, 0, 1
    gcd_val, x1, y1 = extended_gcd(b % a, a)
    x = y1 - (b // a) * x1
    y = x1
    return gcd_val, x, y

def mod_inverse(a, n):
    """Find modular inverse of a mod n using Extended Euclidean Algorithm."""
    gcd_val, x, _ = extended_gcd(a, n)
    if gcd_val != 1:
        raise ValueError("Modular inverse does not exist since gcd(ϕ(n), n) ≠ 1")
    return x % n  # Ensure the result is positive

mu = mod_inverse(lamb, n)

## Encryption

In [9]:
# message
message = 123

# fix an r - should be random and coprime to n but we can use something like hash of all private inputs for zkVM
r = 3

def encrypt(message, r):
    return (pow(g, message, n**2) * pow(r, n, n**2)) % (n ** 2)

ciphertext = encrypt(message, r)
print(f"message: {message}")
print(f"ciphertext: {ciphertext}")

message: 123
ciphertext: 2603914724305477


## Decryption

In [8]:
def L(x):
    return (x - 1) // n

def decrypt(ciphertext):
    return L(pow(ciphertext, lamb, n**2)) * mu % n

decrypted = decrypt(ciphertext)
print(f"decrypted: {decrypted}")

decrypted: 123


## Homomorphic Properties for Private Set Intersection

In [15]:
from sympy import mod_inverse
# The product of two ciphertexts decrypts to the sum of the plaintexts
# D(E(m1, r1) * E(m2, r2)) mod n**n = m1 + m2 mod n

# so if alice makes her E(m1, r1) and gets bob's E(m2, r2)
# she finds the inverse of E(m2, r2) mod n**2
# and multiplies it with E(m1, r1) to get m1 - m2 mod n
# because D(E(m1,r1)^k mod n**2) = km1 mod n; k is -1 for us 

# situation 1: alice and bob have DIFFERENT answers
alice_secret_answer = 123
bob_secret_answer = 456

alice_ciphertext = encrypt(alice_secret_answer, 3)
bob_ciphertext = encrypt(bob_secret_answer, 5)
bob_ciphertext_inv = mod_inverse(bob_ciphertext, n**2)

alice_bob_ciphertext = alice_ciphertext * bob_ciphertext_inv % (n ** 2)
print(f"alice_bob_ciphertext: {alice_bob_ciphertext}")

alice_bob_decrypted = decrypt(alice_bob_ciphertext)
print(f"alice_bob_decrypted: {alice_bob_decrypted}")

# situation 2: alice and bob have the SAME answer
alice_secret_answer = 123
bob_secret_answer = 123

alice_ciphertext = encrypt(alice_secret_answer, 3)
bob_ciphertext = encrypt(bob_secret_answer, 5)
bob_ciphertext_inv = mod_inverse(bob_ciphertext, n**2)

alice_bob_ciphertext = alice_ciphertext * bob_ciphertext_inv % (n ** 2)
print(f"alice_bob_ciphertext: {alice_bob_ciphertext}")

alice_bob_decrypted = decrypt(alice_bob_ciphertext)
print(f"alice_bob_decrypted: {alice_bob_decrypted}")

alice_bob_ciphertext: 1134436537042790
alice_bob_decrypted: 54180764
alice_bob_ciphertext: 1831226466728426
alice_bob_decrypted: 0
